In [164]:
import glob
import pandas as pd
from pathlib import Path

In [54]:
import re

### API

In [173]:
import json
with open('./data/watoo470k/key2folder.json', 'r') as f:
    key2folder = json.load(f)

In [2]:
root = Path('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/')
target_root  = root / 'hwpedia_all'
!ls $target_root

crops  hwpedia_folder  labels


In [3]:
!ls /nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/

T恤		吊带	   斗篷		     牛仔裤  裙子    马甲
休闲裤		大衣	   棉衣_羊羔绒	     皮草    西装    马甲_背心
休闲鞋		夹克_外套  棒球服	     皮衣    西装裤
休闲鞋_setdiff	女衫类	   正装女鞋	     皮鞋    运动裤
包包		小西装	   正装女鞋_setdiff  羽绒服  连衣裙
包包_setdiff	打底裤	   毛衣_绒衣_针织衫  背心    连衣裤
卫衣		抹胸	   牛仔		     衬衫    风衣


In [4]:
%%time
# imgs_list = glob.glob(str(root /'*/*.jpg'))
imgs_list = glob.glob(str(target_root /'hwpedia_folder/T恤/*/*'))

CPU times: user 163 ms, sys: 16.3 ms, total: 179 ms
Wall time: 260 ms


In [158]:
f_list = glob.glob('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/*')

In [160]:
len(f_list)

37

In [162]:
df = pd.DataFrame()
df['path'] = f_list

In [165]:
df['folder'] = df['path'].apply(lambda x: Path(x).parts[-1])

In [169]:
folder2key = {}
for k, f in enumerate(df['folder'].values):
    folder2key[f] = k

In [8]:
imgs_df = pd.read_csv('./data/watoo470k/imgs_watoo470k.csv')

In [7]:
txt_df = pd.read_csv('./data/watoo470k/labels_watoo470k.csv')

In [83]:
txt_df.tail(1).values

array([['/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/labels/fffadbff-226c-4108-9673-f7a41a24a53b.txt',
        'fffadbff-226c-4108-9673-f7a41a24a53b']], dtype=object)

In [84]:
imgs_df.tail(1).values

array([['/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/女衫类/2/fffadbff-226c-4108-9673-f7a41a24a53b.jpg',
        'fffadbff-226c-4108-9673-f7a41a24a53b']], dtype=object)

### 分析img和label的数据

In [76]:
imgs_df['stem'] = imgs_df['path'].apply(lambda x: Path(x).stem)

In [77]:
txt_df['stem'] = txt_df['path'].apply(lambda x: Path(x).stem)

In [88]:
imgs_cross = imgs_df[imgs_df['stem'].isin(txt_df['stem'])]

In [91]:
txt_cross = txt_df[txt_df['stem'].isin(imgs_df['stem'])]

In [80]:
imgs_df.sort_values('stem',inplace=True)
txt_df.sort_values('stem',inplace=True)

In [92]:
len(imgs_df),len(txt_df), len(imgs_cross), len(txt_cross)

(478504, 453275, 453275, 453275)

In [93]:
imgs_cross.sort_values('stem',inplace=True)
txt_cross.sort_values('stem',inplace=True)

/tmp/ipykernel_411860/859653631.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_cross.sort_values('stem',inplace=True)


In [94]:
watoo_df = pd.DataFrame()
watoo_df['img_p'] = imgs_cross['path'].values
watoo_df['label_p'] = txt_cross['path'].values

In [97]:
watoo_df.tail(2).values

array([['/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/牛仔裤/92/fff77871-de9f-4776-9424-f41b48cc0a49.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/labels/fff77871-de9f-4776-9424-f41b48cc0a49.txt'],
       ['/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/女衫类/2/fffadbff-226c-4108-9673-f7a41a24a53b.jpg',
        '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/labels/fffadbff-226c-4108-9673-f7a41a24a53b.txt']],
      dtype=object)

#### 未被检测的数据

In [98]:
imgs_nodet = imgs_df[~imgs_df['stem'].isin(txt_df['stem'])]

In [99]:
len(imgs_nodet)

25229

In [134]:
# imgs_nodet.to_csv('./data/watoo470k/imgs_nodet.csv', index=False)

In [119]:
p = imgs_nodet['path'].values[9]

#### 解析labels txt

In [13]:
p = txt_df.head(1).values[0][0]
!cat $p

2 0.505992 0.516117 0.985353 0.78036 0.810297


In [49]:
p = txt_df.head(10).values[-4][0]
!cat $p

3 0.506 0.923333 0.305333 0.153333 0.707341
2 0.464667 0.59 0.572 0.545333 0.958252


In [74]:
with open(p,'r') as f:
    txt = f.readlines()

In [75]:
txt

['3 0.506 0.923333 0.305333 0.153333 0.707341\n',
 '2 0.464667 0.59 0.572 0.545333 0.958252\n']

In [69]:
for item in txt:
    list1 = re.split(r'\n', item)
    list2 = list1[0].split(' ')
    

In [70]:
list2

['2', '0.464667', '0.59', '0.572', '0.545333', '0.958252']

### 画图

In [29]:
!pwd

/home/chenyi/workspace/myPyModule99/gallery_manage


In [31]:
import sys
sys.path.append('../')

In [36]:
from utils.plots.plots import draw_bbox
from utils.bbox.conversation import *
from utils.display.html import *

### 数据拷贝

In [121]:
import sys
sys.path.append('../')

In [124]:
from utils.data.transfer import copy_data_mkdir, run_task_multithread

#### target path

In [133]:
imgs_nodet.head(2).values

array([[PosixPath('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/卫衣/150/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg'),
        '005a2302-330b-4afc-ae19-5d0e513ee14a',
        '/nas/lichangjian/wardrobe_2/done_/卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg'],
       [PosixPath('/nas/chenyi/datasets_cls/gallery_detect/watoo400k/hwpedia_all/hwpedia_folder/牛仔裤/80/009dcf1b-1bb6-435e-b30c-0678ada49aee.jpg'),
        '009dcf1b-1bb6-435e-b30c-0678ada49aee',
        '/nas/lichangjian/wardrobe_2/done_/牛仔裤/009dcf1b-1bb6-435e-b30c-0678ada49aee.jpg']],
      dtype=object)

In [177]:
sroot = '/nas/lichangjian/wardrobe_2/done_'
troot = '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_kfolder'

In [146]:
!ls /nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/

nodet


In [136]:
imgs_nodet['path'] = imgs_nodet['path'].apply(lambda x: Path(x))

/tmp/ipykernel_411860/2584459815.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['path'] = imgs_nodet['path'].apply(lambda x: Path(x))


In [137]:
imgs_nodet['img_sp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}/{2}'.format(sroot, x.parts[-3], x.name))

/tmp/ipykernel_411860/1833358304.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['img_sp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}/{2}'.format(sroot, x.parts[-3], x.name))


In [178]:
imgs_nodet['img_tp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}_{2}/{3}'.format(troot, folder2key[x.parts[-3]], x.parts[-3], x.name))

/tmp/ipykernel_411860/361264578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  imgs_nodet['img_tp'] = imgs_nodet['path'].apply(lambda x: '{0}/{1}_{2}/{3}'.format(troot, folder2key[x.parts[-3]], x.parts[-3], x.name))


In [179]:
input_list = [list(item) for item in zip(imgs_nodet['img_sp'].values, imgs_nodet['img_tp'].values)]

In [180]:
input_list[0]

['/nas/lichangjian/wardrobe_2/done_/卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg',
 '/nas/chenyi/datasets_cls/gallery_detect/watoo400k/analysis/hwpedia/nodet_kfolder/10_卫衣/005a2302-330b-4afc-ae19-5d0e513ee14a.jpg']

In [181]:
%%time
import shutil
def copy_data_mkdir(plist, mode='copy'):
    ps = Path(plist[0])
    pt = Path(plist[1])
    if not pt.parent.is_dir():
        pt.parent.mkdir(parents=True, exist_ok=True)
    if not pt.is_file():
        if mode=='move':
            Path.rename(ps, pt)
        else:
            shutil.copyfile(ps, pt)
            
run_task_multithread(input_list, copy_data_mkdir)

CPU times: user 5.52 s, sys: 31.3 s, total: 36.9 s
Wall time: 37.1 s
